<a href="https://colab.research.google.com/github/Uriel1201/HelloSQL2.0/blob/main/09_most_friended/query.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#09. Most friended
**Given the following table, return a list of users and their corresponding friend count. Order the result by descending friend count, and in the case of a tie, by ascending user ID. Assume that only unique friendships are displayed.**

In [1]:
!pip install polars
import pandas as pd
import numpy as  np
import polars as pl

In [2]:
data={'user_1':[1,1,1,2],
      'user_2':[2,3,4,3]
     }
pd_friends=pd.DataFrame(data)
pl_friends=pl.LazyFrame(data)

In [3]:
print(f'----PANDAS----\nfriends table:\n{pd_friends}')

----PANDAS----
friends table:
   user_1  user_2
0       1       2
1       1       3
2       1       4
3       2       3


In [7]:
pd_s=(pd.concat([pd_friends['user_1'],
                 pd_friends['user_2']],
                ignore_index=True
         )
        .rename('user_id')
)
pd_friendship=(pd_s.groupby(pd_s)
                    .size()
                    .rename('number_of_friends')
                    .sort_values(ascending=False)
)
print(f'Number of friends by each user:')
pd_friendship

Number of friends by each user:


,number_of_friends
user_id,
1,3
2,2
3,2
4,1


In [8]:
sample=(pl_friends.head(5)
                  .collect()
)
print(f'----POLARS----\nfriends table (SAMPLE):\n{sample}')

----POLARS----
friends table (SAMPLE):
shape: (4, 2)
┌────────┬────────┐
│ user_1 ┆ user_2 │
│ ---    ┆ ---    │
│ i64    ┆ i64    │
╞════════╪════════╡
│ 1      ┆ 2      │
│ 1      ┆ 3      │
│ 1      ┆ 4      │
│ 2      ┆ 3      │
└────────┴────────┘


In [9]:
pl_lf=(pl.sql("""
                 SELECT user_1 as user_id FROM pl_friends
                 UNION ALL
                 SELECT user_2 as user_id FROM pl_friends
              """
          )
         .group_by('*')
         .agg(pl.len()
                .alias('number_of_friends')
          )
         .sort(by='number_of_friends',
               descending=True
          )
).collect()
print(f'Returninh number of friends by each user:')
pl_lf

Returninh number of friends by each user:


user_id,number_of_friends
i64,u32
1,3
2,2
3,2
4,1
